# Initial Study

This notebook identifies PPMI subjects to reproduce the following paper:

<div class="alert alert-block alert-success">
Sterling, Nicholas W, et al. <a href=https://n.neurology.org/content/86/12/1143.short>Stage-dependent loss of cortical gyrification as Parkinson disease “unfolds”.</a> Neurology, vol. 86, no. 12, pp. 1143–1151, Mar. 2016. 
</div>

<img src="demographics.png">

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the Python code."></form>''')



In [5]:
import datetime
print(f'This notebook was run on {datetime.datetime.now()}')

This notebook was run on 2022-07-07 10:24:04.785116


# PPMI metadata download


Let's download the required metadata files from PPMI. You will need to enter your PPMI username and password.

In [ ]:
import os
import os.path as op
import ppmi_downloader

data_dir = 'data'
required_files = ['Age_at_visit.csv',  'Demographics.csv',  
                  'REM_Sleep_Behavior_Disorder_Questionnaire.csv']

missing_files = [x for x in required_files if not op.exists(os.path.join(data_dir, x))]

if len(missing_files) > 0:
    print(f'Downloading missing files: {missing_files}')
    ppmi = ppmi_downloader.PPMIDownloader()
    ppmi.download_metadata(missing_files, destination_dir=data_dir, headless=False, timeout=600)

print(f'The following files are available: {required_files}')